In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def fpreproc_safe(train_test):    
    # convert date to be more useable
    train_test['Date'] = pd.to_datetime(train_test['created'])
    # year is all the same
    #train_test['Year'] = train_test['Date'].dt.year
    train_test['Month'] = train_test['Date'].dt.month
    train_test['Day'] = train_test['Date'].dt.day
    train_test['Wday'] = train_test['Date'].dt.dayofweek
    train_test['Yday'] = train_test['Date'].dt.dayofyear
    train_test['hour'] = train_test['Date'].dt.hour
    train_test = train_test.drop(['Date', 'created'], axis=1)


    # check if this building id/manager id and add worths is 0
    train_test['Zero_building_id'] = train_test['building_id'].apply(lambda x: 1 if x == '0' else 0)
    train_test['Zero_manager_id'] = train_test['manager_id'].apply(lambda x: 1 if x == '0' else 0)
    

    # mess with description meta data
    train_test['desc'] = train_test['description']
    train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
    train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('!<br /><br />', ''))

    string.punctuation.__add__('!!')
    string.punctuation.__add__('(')
    string.punctuation.__add__(')')
    remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

    train_test['desc'] = train_test['desc'].apply(lambda x: x.translate(remove_punct_map))
    train_test['desc_letters_count'] = train_test['description'].apply(lambda x: len(x.strip()))
    train_test['desc_words_count'] = train_test['desc'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))
    train_test['desc_words_length'] = (train_test['desc_letters_count'] / train_test['desc_words_count']).apply(lambda x: 0 if math.isnan(x) or math.isinf(x) else x)

    train_test.drop(['description', 'desc'], axis=1, inplace=True)


    # address
    train_test['address1'] = train_test['display_address']
    train_test['address1'] = train_test['address1'].apply(lambda x: x.lower())

    address_map = {
        'w': 'west',
        'st.': 'street',
        'ave': 'avenue',
        'st': 'street',
        'e': 'east',
        'n': 'north',
        's': 'south'
    }

    def address_map_func(s):
        s = s.split(' ')
        out = []
        for x in s:
            if x in address_map:
                out.append(address_map[x])
            else:
                out.append(x)
        return ' '.join(out)


    train_test['address1'] = train_test['address1'].apply(lambda x: x.translate(remove_punct_map))
    train_test['address1'] = train_test['address1'].apply(lambda x: address_map_func(x))

    new_cols = ['street', 'avenue', 'east', 'west', 'north', 'south']
    for col in new_cols:
        train_test[col] = train_test['address1'].apply(lambda x: 1 if col in x else 0)

    train_test['other_address'] = train_test[new_cols].apply(lambda x: 1 if x.sum() == 0 else 0, axis=1)
    train_test.drop(['display_address', 'street_address'], axis=1, inplace=True)


    # features (better not to lower)
    train_test['features_count'] = train_test['features'].apply(lambda x: len(x))
    train_test['features2'] = train_test['features']
    train_test['features2'] = train_test['features2'].apply(lambda x: ' '.join([''.join(i.replace('_',' ').replace('-',' ').split(' ')) for i in x]))

    c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
    c_vect.fit(train_test['features2'])

    c_vect_sparse_1 = c_vect.transform(train_test['features2'])
    c_vect_sparse1_cols = c_vect.get_feature_names()
    train_test.drop(['features', 'features2'], axis=1, inplace=True)


    # photos
    train_test['photos_count'] = train_test['photos'].apply(lambda x: len(x))
    train_test.drop(['photos'], axis=1, inplace=True)


    # convert ['building_id', 'manager_id', 'address1'] to enumerated labels
    categoricals = [x for x in train_test.columns if train_test[x].dtype == 'object']
    for feat in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_test[feat].values))
        train_test[feat] = lbl.transform(list(train_test[feat].values))


    # convert bed/bath number to enumeration
    train_test['bathrooms_cat'] = train_test['bathrooms'].apply(lambda x: str(x))
    train_test['bathrooms_cat'], labels = pd.factorize(train_test['bathrooms_cat'].values, sort=True)
    train_test.drop('bathrooms', axis=1, inplace=True)
    train_test['bedroom_cat'], labels = pd.factorize(train_test['bedrooms'].values, sort=True)
    train_test.drop('bedrooms', axis=1, inplace=True)

    
    # transform managers and building ids
    train_test['manager_id'] = train_test['manager_id'].apply(lambda x: str(x))
    train_test['manager_id'], labels = pd.factorize(train_test['manager_id'].values, sort=True)
    train_test['building_id'] = train_test['building_id'].apply(lambda x: str(x))
    train_test['building_id'], labels = pd.factorize(train_test['building_id'].values, sort=True)
    

    # transform price
    bc_price, tmp = boxcox(train_test.price)
    train_test['bc_price'] = bc_price


    # add sparse
    train_test_cv1_sparse = sparse.hstack((train_test, c_vect_sparse_1)).tocsr()


    # add feature names
    features = list(train_test.columns) + ['sparse_' + vect_name for vect_name in c_vect_sparse1_cols]
    ddata = pd.DataFrame(np.array(train_test_cv1_sparse.todense()), columns=features)
    return ddata

In [3]:
def fpreproc_leaky(dtrain, dtest, param):
    train, test = dtrain.data, dtest.data
    ntrain = train.shape[0]
    train_test = pd.concat((train, test), axis=0).reset_index(drop=True)
    y_train = dtrain.labels
    
    # add manager worths
    manager_ids = np.unique(train.manager_id)
    manager_worths = {}
    for manager_id in manager_ids:
        interests = y_train[train.manager_id == manager_id].apply(lambda x: 2 - x)
        if len(interests) > 5:
            manager_worths[manager_id] = sum(interests) / len(interests)
    average_interest = y_train.apply(lambda x: 2 - x)
    average_manager_worth = 0 #sum(average_interest) / len(average_interest)
    train_test['manager_worths'] = train_test['manager_id'].apply(lambda x: manager_worths[x] if x in manager_worths else average_manager_worth)

    
    # add building worths
    building_ids = np.unique(train.building_id)
    building_worths = {}
    for building_id in building_ids:
        interests = y_train[train.building_id == building_id].apply(lambda x: 2 - x)
        if len(interests) > 5:
            building_worths[building_id] = sum(interests) / len(interests)
    average_interest = y_train.apply(lambda x: 2 - x)
    average_building_worth = 0 #sum(average_interest) / len(average_interest)
    train_test['building_worths'] = train_test['building_id'].apply(lambda x: building_worths[x] if x in building_worths else average_building_worth)

    
    # add price by area
    lat_long_price = train_test[['latitude', 'longitude', 'price']]
    remove_outliers = (np.abs(stats.zscore(lat_long_price)) < 0.15).all(axis=1)
    lat_long_price = lat_long_price[remove_outliers]
    lat_max, lat_min = max(lat_long_price.latitude), min(lat_long_price.latitude)
    long_max, long_min = max(lat_long_price.longitude), min(lat_long_price.longitude)
    lat_scale, long_scale = lat_max - lat_min, long_max - long_min
    costs = np.zeros((100,100))
    num_listings = np.zeros((100,100))
    for lat, long, price in lat_long_price.values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        costs[scaled_lat][scaled_long] += price
        num_listings[scaled_lat][scaled_long] += 1

    price_by_area = []
    for lat, long, price in train_test[['latitude', 'longitude', 'price']].values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        if scaled_lat < 0 or scaled_lat >= 100 or scaled_long < 0 or scaled_long >= 100:
            price_by_area.append(0)
        elif num_listings[scaled_lat][scaled_long] > 8:
            price_by_area.append(price - (costs[scaled_lat][scaled_long] / num_listings[scaled_lat][scaled_long]))
        else:
            cost = 0
            num = 0
            for i in range(scaled_lat - 1, scaled_lat + 2):
                for j in range(scaled_long - 1, scaled_long + 2):
                    if i > 0 and i < 100 and j >= 0 and j < 100:
                        cost += costs[i][j]
                        num += num_listings[i][j]
            if num > 8:
                price_by_area.append(price - (cost / num))
            else:
                price_by_area.append(0)

    train_test['price_by_area'] = price_by_area
    
    
    # try adding real - predicted price
    # Try to predict price for a listing and add real_price - expected_price as a feature
    # train on [exclusive, byowner, nofee] and #bed, #bath, display address
    features_to_use = ['bathrooms_cat', 'bedroom_cat', 'street', 'avenue', 'east', 'west', 'north', 'south', 'address1', 'other_address', 'building_worths', 'manager_worths']
    feature_to_predict = 'price'
    params = {
        'objective': 'reg:linear',
        'booster':'gblinear',
        'lambda': 0,
        'lambda_bias' : 0,
        'alpha': 0.2
    }
    prices = train_test[feature_to_predict]
    remove_outliers = np.abs(prices-prices.mean())<=(3*prices.std())
    dtrain = xgb.DMatrix(data=train_test[remove_outliers][features_to_use], label=train_test[remove_outliers][feature_to_predict])

    bst = xgb.cv(params, dtrain, 10000, 4, early_stopping_rounds=50, verbose_eval=200)
    best_rounds = np.argmin(bst['test-rmse-mean'])
    print (bst['test-rmse-mean'][best_rounds])
    bst = xgb.train(params, dtrain, best_rounds)
    dtrain = xgb.DMatrix(data=train_test[features_to_use])
    expected_price = bst.predict(dtrain)
    train_test['real_minus_expected_price'] = train_test[feature_to_predict] - expected_price
    train_test['real_over_expected_price'] = train_test[feature_to_predict] / expected_price
    
    
    # remove extra
    train_test.drop('price', axis=1, inplace=True)
    
    dtrain = xgb.DMatrix(data=train_test[:ntrain], label=y_train)
    dtest = xgb.DMatrix(data=train_test[ntrain:], label=dtest.labels)
    return dtrain, dtest, param

In [4]:
%%time
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])
    
SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

train = pd.read_json('data/train.json')
y_train = train['interest_level'].apply(lambda x: y_map[x]).reset_index(drop=True)
train = train.drop(['listing_id', 'interest_level'], axis=1)

dtrain = FakeDMatrix(data=fpreproc_safe(train), labels=y_train)
train = None
y_train = None

CPU times: user 8.65 s, sys: 890 ms, total: 9.54 s
Wall time: 11.1 s


In [ ]:
%%time
bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200, fpreproc=fpreproc_leaky)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

[0]	train-rmse:2006.05+20.5056	test-rmse:2005.42+65.7666
[200]	train-rmse:1639.97+20.3985	test-rmse:1639.73+61.0016
1639.72482325
[0]	train-rmse:1980.51+54.3064	test-rmse:1979.73+72.9905
[200]	train-rmse:1640.28+21.246	test-rmse:1640.81+63.8306
1640.81063825
[0]	train-rmse:1954.28+28.8797	test-rmse:1955.18+67.4425
[200]	train-rmse:1640.12+22.0148	test-rmse:1640.17+65.7121
1640.16406275
[0]	train-mlogloss:1.09143+2.37908e-05	test-mlogloss:1.09175+1.08423e-05


In [ ]:
%%time
test = pd.read_json('data/test.json')
listing_id = test.listing_id.values
test = test.drop('listing_id', axis=1)
dtest = FakeDMatrix(data=fpreproc_safe(test))
test = None

dtrain_final, dtest_final, _ = fpreproc_leaky(dtrain, dtest, None)
dtrain = None
dtest = None

In [ ]:
bst = xgb.train(params, dtrain_final, best_rounds)
dtrain_final = None

preds = bst.predict(dtest_final)
dtest_final = None

preds = pd.DataFrame(preds)
preds.columns = ['high', 'medium', 'low']
preds['listing_id'] = listing_id
preds.to_csv('my_preds.csv', index=None)

importance = bst.get_fscore()
feature_importance = [(feature, (importance[feature]) if feature in importance else 0) for feature in dtest_final.feature_names]
print (sorted(feature_importance, key=lambda x: -x[1]))